# Multirotor Dynamic Model

In [4]:
import casadi as ca
import numpy as np
import sys
sys.path.insert(0, '../python/pyecca')
import pyecca.lie.so3 as so3

Here we will develop a simple thrust model for the propeller.

In [5]:
def thrust(throttle, rho, r, V, kV, CT):
    """
    throttle: 0-1 controls voltage, []
    rho: density of air, [kg/m^3
    r: radius of propeller, [m]
    V: voltage of battery [V]
    kV: motor kV constant, [rpm/V]
    """
    omega = throttle*V*kV*(2*ca.pi/60)
    q = 0.5*rho*(omega*r)**2
    s = ca.pi*r**2
    return CT*q*s

thrust(throttle=1, rho=1.225, r=0.05, V=11.1, kV=1550, CT=1.0e-2)

0.39039349535108125

In [13]:

# state space model
#x: state: (position, orientation, velocity (body frame), angular velocity (body frame))
#u: control
u = ca.SX.sym('u', 4)

#y: output, just state for now

l_arm = 0.5
arm_angles = [45, -135, -34, 135]
for arm_angle in arm_angles:
    print(arm_angle)

thrust1 = thrust(throttle=throttle1, rho=1.225, r=0.05, V=11.1, kV=1550, CT=1.0e-2)

F_b = ca.vertcat(0, 0, -thrust1)
F_b

45
-135
-34
135


SX(@1=0, [@1, @1, (-(0.00785398*(0.01*(0.6125*sq((0.05*(0.10472*(1550*(11.1*throttle2)))))))))])

# Kinematics

Consider the dynamics

v = $\begin{bmatrix}
a
\end{bmatrix}$

In [16]:
def SO3_adjoint(V):
    v = V[0:3]
    w = V[3:6]